In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    eloss = -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a'): 3
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a'): 4
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a'): 5
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a'): 6
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    #################################################### s'', Q' = ~env(s', ~a'): 7
    next_actions_logits = Act(states=next_states_logits, hidden_size=hidden_size, action_size=action_size, 
                              reuse=True)
    next_states_logits, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                                   action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits += tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/7, # GAN
                                                                    labels=tf.zeros_like(rates))) # min
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/7, # GAN
                                                                     labels=tf.ones_like(rates))) # max
#     ###################################################### Q(s,a)= r + Q'(s',a') # max
#     ###################################################### ~Q(s,~a)= r # min
#     ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
#     targetQs = rewards + (gamma * aQs2_logits)
#     eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:23.0000 R:23.0000 rate:0.0460 aloss:0.7023 eloss:0.9209 aloss2:1.3352 exploreP:0.9977
Episode:1 meanR:22.5000 R:22.0000 rate:0.0440 aloss:0.7065 eloss:0.8828 aloss2:1.3467 exploreP:0.9956
Episode:2 meanR:25.3333 R:31.0000 rate:0.0620 aloss:0.6941 eloss:0.8805 aloss2:1.3454 exploreP:0.9925
Episode:3 meanR:24.7500 R:23.0000 rate:0.0460 aloss:0.6878 eloss:0.9573 aloss2:1.3158 exploreP:0.9902
Episode:4 meanR:30.2000 R:52.0000 rate:0.1040 aloss:0.6947 eloss:0.8326 aloss2:1.3611 exploreP:0.9852
Episode:5 meanR:27.8333 R:16.0000 rate:0.0320 aloss:0.6921 eloss:0.8257 aloss2:1.3584 exploreP:0.9836
Episode:6 meanR:25.8571 R:14.0000 rate:0.0280 aloss:0.6965 eloss:0.8339 aloss2:1.3562 exploreP:0.9822
Episode:7 meanR:27.2500 R:37.0000 rate:0.0740 aloss:0.6893 eloss:0.8302 aloss2:1.3531 exploreP:0.9787
Episode:8 meanR:25.7778 R:14.0000 rate:0.0280 aloss:0.6929 eloss:0.8614 aloss2:1.3409 exploreP:0.9773
Episode:9 meanR:27.7000 R:45.0000 rate:0.0900 aloss:0.6875 eloss:0.8313 aloss2:1.3

Episode:80 meanR:25.6914 R:16.0000 rate:0.0320 aloss:0.6845 eloss:0.2280 aloss2:1.6926 exploreP:0.8140
Episode:81 meanR:25.7439 R:30.0000 rate:0.0600 aloss:0.6878 eloss:0.1734 aloss2:1.7420 exploreP:0.8116
Episode:82 meanR:25.8916 R:38.0000 rate:0.0760 aloss:0.6822 eloss:0.1905 aloss2:1.7329 exploreP:0.8086
Episode:83 meanR:25.7381 R:13.0000 rate:0.0260 aloss:0.6872 eloss:0.1481 aloss2:1.7720 exploreP:0.8075
Episode:84 meanR:25.6941 R:22.0000 rate:0.0440 aloss:0.6723 eloss:0.2105 aloss2:1.7095 exploreP:0.8058
Episode:85 meanR:25.5698 R:15.0000 rate:0.0300 aloss:0.6723 eloss:0.1755 aloss2:1.7334 exploreP:0.8046
Episode:86 meanR:25.4598 R:16.0000 rate:0.0320 aloss:0.6834 eloss:0.1431 aloss2:1.7723 exploreP:0.8033
Episode:87 meanR:26.3409 R:103.0000 rate:0.2060 aloss:0.6784 eloss:0.1287 aloss2:1.7961 exploreP:0.7952
Episode:88 meanR:26.7079 R:59.0000 rate:0.1180 aloss:0.6763 eloss:0.1476 aloss2:1.7777 exploreP:0.7906
Episode:89 meanR:26.8444 R:39.0000 rate:0.0780 aloss:0.6818 eloss:0.0966

Episode:159 meanR:41.4500 R:51.0000 rate:0.1020 aloss:0.6762 eloss:-1.1823 aloss2:5.0892 exploreP:0.5685
Episode:160 meanR:42.0200 R:93.0000 rate:0.1860 aloss:0.6837 eloss:-1.2204 aloss2:5.1845 exploreP:0.5633
Episode:161 meanR:42.1800 R:49.0000 rate:0.0980 aloss:0.6808 eloss:-1.2285 aloss2:5.1264 exploreP:0.5606
Episode:162 meanR:42.4000 R:39.0000 rate:0.0780 aloss:0.6804 eloss:-1.3294 aloss2:5.3618 exploreP:0.5585
Episode:163 meanR:42.7400 R:47.0000 rate:0.0940 aloss:0.6848 eloss:-1.3178 aloss2:5.4155 exploreP:0.5559
Episode:164 meanR:42.5600 R:29.0000 rate:0.0580 aloss:0.6770 eloss:-1.3543 aloss2:5.5667 exploreP:0.5543
Episode:165 meanR:42.2400 R:20.0000 rate:0.0400 aloss:0.7100 eloss:-1.1010 aloss2:4.8544 exploreP:0.5532
Episode:166 meanR:42.8000 R:67.0000 rate:0.1340 aloss:0.6797 eloss:-1.3934 aloss2:5.5414 exploreP:0.5496
Episode:167 meanR:42.7100 R:15.0000 rate:0.0300 aloss:0.6856 eloss:-1.4451 aloss2:5.5654 exploreP:0.5488
Episode:168 meanR:42.7700 R:25.0000 rate:0.0500 aloss:0

Episode:238 meanR:44.2700 R:16.0000 rate:0.0320 aloss:0.7717 eloss:-3.3953 aloss2:9.6494 exploreP:0.4060
Episode:239 meanR:44.0100 R:13.0000 rate:0.0260 aloss:0.7435 eloss:-3.4079 aloss2:9.6504 exploreP:0.4054
Episode:240 meanR:43.0800 R:17.0000 rate:0.0340 aloss:0.8162 eloss:-3.0915 aloss2:8.9204 exploreP:0.4048
Episode:241 meanR:42.9600 R:14.0000 rate:0.0280 aloss:0.7574 eloss:-3.4747 aloss2:10.0201 exploreP:0.4042
Episode:242 meanR:42.5900 R:32.0000 rate:0.0640 aloss:0.8035 eloss:-3.2685 aloss2:9.3170 exploreP:0.4030
Episode:243 meanR:41.6600 R:19.0000 rate:0.0380 aloss:0.8119 eloss:-3.2436 aloss2:9.2763 exploreP:0.4022
Episode:244 meanR:41.5600 R:17.0000 rate:0.0340 aloss:0.7917 eloss:-3.0827 aloss2:9.0234 exploreP:0.4015
Episode:245 meanR:41.4000 R:13.0000 rate:0.0260 aloss:0.7669 eloss:-3.2460 aloss2:9.4164 exploreP:0.4010
Episode:246 meanR:41.3700 R:13.0000 rate:0.0260 aloss:0.7821 eloss:-3.4573 aloss2:9.7014 exploreP:0.4005
Episode:247 meanR:41.1900 R:22.0000 rate:0.0440 aloss:

Episode:316 meanR:13.6400 R:8.0000 rate:0.0160 aloss:0.8141 eloss:-4.1039 aloss2:11.4504 exploreP:0.3676
Episode:317 meanR:13.6100 R:10.0000 rate:0.0200 aloss:0.8733 eloss:-3.8787 aloss2:10.9316 exploreP:0.3673
Episode:318 meanR:13.5500 R:9.0000 rate:0.0180 aloss:0.9607 eloss:-3.6459 aloss2:10.7966 exploreP:0.3670
Episode:319 meanR:13.4800 R:10.0000 rate:0.0200 aloss:0.7886 eloss:-4.0738 aloss2:11.3942 exploreP:0.3666
Episode:320 meanR:13.4100 R:10.0000 rate:0.0200 aloss:1.0538 eloss:-3.5666 aloss2:10.3985 exploreP:0.3662
Episode:321 meanR:13.4000 R:14.0000 rate:0.0280 aloss:1.0914 eloss:-3.3024 aloss2:9.9084 exploreP:0.3657
Episode:322 meanR:13.4200 R:15.0000 rate:0.0300 aloss:0.8812 eloss:-3.7713 aloss2:10.8296 exploreP:0.3652
Episode:323 meanR:13.3900 R:13.0000 rate:0.0260 aloss:0.8645 eloss:-3.8104 aloss2:11.0355 exploreP:0.3647
Episode:324 meanR:13.3000 R:10.0000 rate:0.0200 aloss:0.8712 eloss:-3.6673 aloss2:10.9177 exploreP:0.3644
Episode:325 meanR:13.2400 R:11.0000 rate:0.0220 a

...

Episode:2502 meanR:228.7400 R:274.0000 rate:0.5480 aloss:0.1513 eloss:812.8744 aloss2:72.1720 exploreP:0.0100
Episode:2503 meanR:230.2200 R:339.0000 rate:0.6780 aloss:0.1496 eloss:800.6001 aloss2:71.3913 exploreP:0.0100
Episode:2504 meanR:230.2600 R:240.0000 rate:0.4800 aloss:0.1613 eloss:813.7571 aloss2:71.3313 exploreP:0.0100
Episode:2505 meanR:230.7000 R:216.0000 rate:0.4320 aloss:0.1443 eloss:726.7202 aloss2:71.4856 exploreP:0.0100
Episode:2506 meanR:231.4300 R:254.0000 rate:0.5080 aloss:0.1375 eloss:1445.6254 aloss2:71.1572 exploreP:0.0100
Episode:2507 meanR:231.3000 R:182.0000 rate:0.3640 aloss:0.1403 eloss:789.3307 aloss2:71.1605 exploreP:0.0100
Episode:2508 meanR:231.3100 R:211.0000 rate:0.4220 aloss:0.1615 eloss:956.0480 aloss2:70.7209 exploreP:0.0100
Episode:2509 meanR:231.5200 R:211.0000 rate:0.4220 aloss:0.1547 eloss:812.6641 aloss2:71.1042 exploreP:0.0100
Episode:2510 meanR:230.6900 R:226.0000 rate:0.4520 aloss:0.1783 eloss:860.5692 aloss2:71.6772 exploreP:0.0100
Episode:2

Episode:2576 meanR:247.0600 R:291.0000 rate:0.5820 aloss:0.1423 eloss:5093.7305 aloss2:61.3866 exploreP:0.0100
Episode:2577 meanR:247.4500 R:234.0000 rate:0.4680 aloss:0.1530 eloss:4776.2832 aloss2:61.5217 exploreP:0.0100
Episode:2578 meanR:247.9900 R:336.0000 rate:0.6720 aloss:0.1337 eloss:5391.8052 aloss2:61.1972 exploreP:0.0100
Episode:2579 meanR:248.4300 R:231.0000 rate:0.4620 aloss:0.1527 eloss:5125.4141 aloss2:60.9159 exploreP:0.0100
Episode:2580 meanR:248.8300 R:224.0000 rate:0.4480 aloss:0.1358 eloss:5218.8765 aloss2:60.2871 exploreP:0.0100
Episode:2581 meanR:248.8200 R:190.0000 rate:0.3800 aloss:0.1616 eloss:23084.9688 aloss2:58.5475 exploreP:0.0100
Episode:2582 meanR:249.1600 R:230.0000 rate:0.4600 aloss:0.1434 eloss:29068.6777 aloss2:54.3191 exploreP:0.0100
Episode:2583 meanR:248.5800 R:201.0000 rate:0.4020 aloss:0.1290 eloss:24356.0742 aloss2:53.1399 exploreP:0.0100
Episode:2584 meanR:248.6800 R:205.0000 rate:0.4100 aloss:0.1451 eloss:22378.4199 aloss2:53.1692 exploreP:0.01

Episode:2650 meanR:238.4100 R:281.0000 rate:0.5620 aloss:0.1502 eloss:-89.2710 aloss2:94.3064 exploreP:0.0100
Episode:2651 meanR:238.1600 R:234.0000 rate:0.4680 aloss:0.1548 eloss:-96.6933 aloss2:102.5481 exploreP:0.0100
Episode:2652 meanR:238.3400 R:293.0000 rate:0.5860 aloss:0.1621 eloss:-104.2172 aloss2:112.2417 exploreP:0.0100
Episode:2653 meanR:238.6400 R:276.0000 rate:0.5520 aloss:0.1747 eloss:-114.5406 aloss2:122.8997 exploreP:0.0100
Episode:2654 meanR:237.8300 R:232.0000 rate:0.4640 aloss:0.1634 eloss:-123.3552 aloss2:129.4321 exploreP:0.0100
Episode:2655 meanR:236.8700 R:262.0000 rate:0.5240 aloss:0.1826 eloss:-135.8694 aloss2:143.9709 exploreP:0.0100
Episode:2656 meanR:237.4800 R:242.0000 rate:0.4840 aloss:0.2076 eloss:-148.4644 aloss2:165.7085 exploreP:0.0100
Episode:2657 meanR:237.4500 R:206.0000 rate:0.4120 aloss:0.2014 eloss:-134.5880 aloss2:197.2917 exploreP:0.0100
Episode:2658 meanR:236.9900 R:254.0000 rate:0.5080 aloss:0.1899 eloss:-144.7466 aloss2:154.1760 exploreP:0.

Episode:2724 meanR:242.2400 R:377.0000 rate:0.7540 aloss:0.1141 eloss:-195.7677 aloss2:8244.8428 exploreP:0.0100
Episode:2725 meanR:244.4300 R:431.0000 rate:0.8620 aloss:0.1201 eloss:-195.5517 aloss2:4841.9268 exploreP:0.0100
Episode:2726 meanR:244.8300 R:225.0000 rate:0.4500 aloss:0.1159 eloss:-191.4749 aloss2:4032.5405 exploreP:0.0100
Episode:2727 meanR:245.6000 R:230.0000 rate:0.4600 aloss:0.1164 eloss:-107.4008 aloss2:2144.8513 exploreP:0.0100
Episode:2728 meanR:247.6400 R:434.0000 rate:0.8680 aloss:0.1251 eloss:5337.7295 aloss2:14072.1289 exploreP:0.0100
Episode:2729 meanR:246.8100 R:183.0000 rate:0.3660 aloss:0.1184 eloss:15427.5850 aloss2:586.8099 exploreP:0.0100
Episode:2730 meanR:246.4300 R:207.0000 rate:0.4140 aloss:0.1187 eloss:10338.2656 aloss2:533.7205 exploreP:0.0100
Episode:2731 meanR:246.5400 R:260.0000 rate:0.5200 aloss:0.1179 eloss:7470.8320 aloss2:530.9679 exploreP:0.0100
Episode:2732 meanR:246.4800 R:204.0000 rate:0.4080 aloss:0.1280 eloss:4127.9565 aloss2:248.3956 

Episode:2798 meanR:261.7100 R:299.0000 rate:0.5980 aloss:0.1131 eloss:498.9297 aloss2:374.6387 exploreP:0.0100
Episode:2799 meanR:262.0000 R:286.0000 rate:0.5720 aloss:0.1081 eloss:1647.2329 aloss2:373.9086 exploreP:0.0100
Episode:2800 meanR:260.9300 R:183.0000 rate:0.3660 aloss:0.1248 eloss:1035.5869 aloss2:381.2545 exploreP:0.0100
Episode:2801 meanR:260.8800 R:222.0000 rate:0.4440 aloss:0.1194 eloss:8429.5742 aloss2:435.7885 exploreP:0.0100
Episode:2802 meanR:262.5300 R:363.0000 rate:0.7260 aloss:0.1301 eloss:7999.0649 aloss2:1083.3889 exploreP:0.0100
Episode:2803 meanR:262.4000 R:196.0000 rate:0.3920 aloss:0.1296 eloss:516.8722 aloss2:589.0449 exploreP:0.0100
Episode:2804 meanR:261.9000 R:210.0000 rate:0.4200 aloss:0.1180 eloss:-125.1958 aloss2:456.7502 exploreP:0.0100
Episode:2805 meanR:262.0900 R:227.0000 rate:0.4540 aloss:0.1187 eloss:-235.0502 aloss2:465.3570 exploreP:0.0100
Episode:2806 meanR:263.0000 R:277.0000 rate:0.5540 aloss:0.1308 eloss:745.9971 aloss2:481.1590 exploreP:0

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
